In [10]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

batch_size = 512
count = batch_size*100
epochs = 100

X = np.float32(np.random.uniform(-1, 1, size=(count, 3)))
y = np.zeros((count, 1), dtype=np.float32)
for i in range(count):
  y[i] = (X[i,0]*X[i,0] + X[i,1]*X[i,1] + X[i,2]*X[i,2]) ** 0.5 - 0.75

print(X)
print(y)

X = torch.from_numpy(X)
y = torch.from_numpy(y)
train_data = torch.utils.data.TensorDataset(X, y)

train_dataloader = DataLoader(train_data, batch_size=batch_size)

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class SinActivation(torch.nn.Module):
    def __init__(self):
        super(SinActivation, self).__init__()
        return
    def forward(self, x):
        return torch.sin(30.0*x)

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, layers, size):
        super().__init__()
        self.flatten = nn.Flatten()
        
        if (layers == 3):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )
        elif (layers == 4):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )
        elif (layers == 5):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )
        elif (layers == 6):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #if batch % 100 == 0:
        #    loss, current = loss.item(), (batch + 1) * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

[[ 0.03480644  0.25782976 -0.8950824 ]
 [-0.18608221 -0.7740489  -0.12990074]
 [-0.19434917  0.0229037  -0.36781996]
 ...
 [ 0.51331323  0.19917841 -0.86242265]
 [ 0.5535587   0.19436698  0.97343224]
 [ 0.23317629  0.42509586  0.36259106]]
[[ 0.18212673]
 [ 0.05663035]
 [-0.33336142]
 ...
 [ 0.27319857]
 [ 0.38656327]
 [-0.14456633]]
Using cuda device


In [13]:
model = NeuralNetwork(5, 256).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [14]:
%%time
epochs = 100
for t in range(epochs):
    train(train_dataloader, model, loss_fn, optimizer)

CPU times: user 36 s, sys: 85.2 ms, total: 36.1 s
Wall time: 36 s


In [15]:
size = len(train_dataloader.dataset)
num_batches = len(train_dataloader)
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): SinActivation()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): SinActivation()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): SinActivation()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): SinActivation()
    (8): Linear(in_features=256, out_features=256, bias=True)
    (9): SinActivation()
    (10): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [16]:
%%time
with torch.no_grad():
  i = 0
  for X, y in train_dataloader:
    X, y = X.to(device), y.to(device)
    pred = model(X)

CPU times: user 323 ms, sys: 12.1 ms, total: 335 ms
Wall time: 333 ms
